In [2]:
import gym
import math
import random
import sklearn
import time
import matplotlib.pyplot as plt

In [3]:
env = gym.make('MountainCar-v0', render_mode="rgb_array").env

In [4]:
print(env.action_space)

Discrete(3)


In [5]:
from sklearn.preprocessing import KBinsDiscretizer
n_bins = ( 20 , 20 )
lower_bounds = [ env.observation_space.low[0], env.observation_space.high[0] ]
upper_bounds = [ env.observation_space.high[1], env.observation_space.low[1] ]

def discretizer( position, velocity ):
    """Convert continues state intro a discrete state"""
    est = KBinsDiscretizer(n_bins=n_bins, encode='ordinal', strategy='uniform')
    est.fit([lower_bounds, upper_bounds ])
    return tuple(map(int,est.transform([[position, velocity]])[0]))


In [6]:
print(*env.reset()[0])
print(discretizer(*env.reset()[0]))
print(random.uniform(0, 1))

-0.55472517 0.0
(10, 2)
0.3028033932143471


In [7]:
import numpy as np
env.reset() # reset environment to a new, random state
env_discretized = discretizer(*env.reset()[0])

print("Action Space {}".format(env.action_space.n))
print("State Space {}".format(env.observation_space))
q_table = np.zeros([env_discretized[0]*env_discretized[1],env.action_space.n])
q_table.shape

Action Space 3
State Space Box([-1.2  -0.07], [0.6  0.07], (2,), float32)


(22, 3)

In [8]:
q_table = np.zeros(n_bins + (env.action_space.n,))
q_table.shape

(20, 20, 3)

In [9]:
#%%time
"""Training the agent"""

import random
from IPython.display import clear_output

# Hyperparameters
alpha = 0.1
gamma = 0.6
epsilon = 0.1

# For plotting metrics
all_epochs = []
all_penalties = []
max_velocity = 0

for i in range(1, 5000):
    
    k=0
    state = discretizer(*env.reset()[0])
    epochs, penalties, reward = 0, 0, 0
    done = False
    
    while k<=200:
        if random.uniform(0, 1) < epsilon:
            action = env.action_space.sample() # Explore action space
            #print('ok')
        else:
            state = discretizer(*state)
            action = np.argmax(q_table[state]) # Exploit learned values
        #(q_table)
        next_state, reward, done, info, test = env.step(action) 
        if next_state[1] > max_velocity:
            max_velocity = next_state[1]
            reward += 20

        state = discretizer(*state)
        #print('state',state)
        old_value = q_table[state, action]
        next_max = np.max(q_table[discretizer(*next_state)])
        
        new_value = (1 - alpha) * old_value + alpha * (reward + gamma * next_max)
        q_table[state, action] = new_value

        if reward == -1:
            penalties += 1

        state = next_state
        epochs += 1
        k+=1
    #print("couo")
    if i % 100 == 0:
        clear_output(wait=True)
        print(f"Episode: {i}")

print("Training finished.\n")

Episode: 4900
Training finished.



In [10]:
total_epochs, total_penalties = 0, 0
episodes = 100
env = gym.make('MountainCar-v0', render_mode="human").env
for _ in range(episodes):
    state = discretizer(*env.reset()[0])
    epochs, penalties, reward = 0, 0, 0
    
    done = False
    
    while not done:
        action = np.argmax(q_table[state])
        next_state, reward, done, info, test = env.step(action) 

        if reward == -1:
            penalties += 1

        epochs += 1

    total_penalties += penalties
    total_epochs += epochs

print(f"Results after {episodes} episodes:")
print(f"Average timesteps per episode: {total_epochs / episodes}")
print(f"Average penalties per episode: {total_penalties / episodes}")

KeyboardInterrupt: 

In [ ]:
choose nbin?

array([[0., 0., 0.],
       [0., 0., 0.],
       [0., 0., 0.],
       ...,
       [0., 0., 0.],
       [0., 0., 0.],
       [0., 0., 0.]])